In [5]:

import os
import numpy as np
import pandas as pd
import warnings
import tensorflow as tf
from keras.models import Model, Sequential
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D, BatchNormalization, GlobalMaxPooling2D, Activation, GlobalMaxPooling1D, Conv1D
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
from keras.wrappers.scikit_learn import KerasClassifier

np.random.seed(42)
warnings.filterwarnings('ignore')

os.environ['OMP_NUM_THREADS'] = '4'

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
submission = pd.read_csv('data/sample_submission.csv')


class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
parag_lens = [250, 400]
vocalens = [20001, 20001]
eb_lens = [200, 100]


y_train = train[class_names].values#[:num_lines]


X_train = np.load("temp_res/train_mulinp250400.npy")
X_test = np.load("temp_res/test_mulinp250400.npy")


In [6]:

def split_inp(inp_splits, inp):
    sub_inps = [[] for _ in inp_splits]
    for row in inp:
        left = 0
        for i in range(len(inp_splits)):
            right = left + inp_splits[i]
            sub_inps[i].append(row[left:right])
            left = right
    x = []
    for sub in sub_inps:
        x.append(np.array(sub))
    return x



def create_cnn(filter_sizes, filter_nums, idp_size, dp, solver='adam'):
    abs_features = []
    inputs = []
    for i in range(len(parag_lens)):
        inp = Input(shape=(parag_lens[i],))
        inputs.append(inp)
        x = Embedding(vocalens[i], eb_lens[i], input_length=parag_lens[i], embeddings_initializer="uniform")(inp)
        flat_feature = base_line_flat(x, parag_lens[i], filter_sizes, eb_lens[i])
        dilated_feature = base_line_dilated(x, parag_lens[i], filter_nums, eb_lens[i])
        abs_feature = Concatenate(axis=-1)([flat_feature, dilated_feature])
        abs_features.append(abs_feature)
    f_ttl = Concatenate(axis=1)(abs_features)
    print(f_ttl.shape)
    f_ttl = Dropout(dp)(f_ttl)
    outs = []
    for _ in range(0,6):
        indp_layer = Dense(idp_size)(f_ttl)
        indp_layer = BatchNormalization()(indp_layer)
        indp_layer = Activation("relu")(indp_layer)
        indp_layer = Dropout(dp)(indp_layer)
        indp_layer = Dense(idp_size//2)(indp_layer)
        indp_layer = BatchNormalization()(indp_layer)
        indp_layer = Activation("relu")(indp_layer)
        indp_layer = Dropout(dp)(indp_layer)
        outs.append(Dense(1, activation="sigmoid")(indp_layer))
    outp =  Concatenate(axis=1)(outs)

    model = Model(inputs=inputs, outputs=outp)
    model.compile(loss='binary_crossentropy',
                  optimizer=solver,
                  metrics=['accuracy'])

    model.summary()

    return model

def base_line_flat(inp, maxlen, filter_sizes, embed_size):
    """
    Define all tunnable params here for model building
    """
    x = Reshape((maxlen, embed_size, 1))(inp)
    outs = []
    for size in filter_sizes:
        conv = Conv2D(size * 10, kernel_size=(size, embed_size), kernel_initializer='normal')(x)
        conv = BatchNormalization()(conv)
        conv = Activation("elu")(conv)
        #conv = Dropout(dp)(conv)
        maxpool = MaxPool2D(pool_size=(maxlen - size + 1, 1))(conv)
        outs.append(maxpool)
    if len(outs) > 1:
        z = Concatenate(axis=-1)(outs)
    else:
        z = outs[0]
    z = Flatten()(z)

    return z

def base_line_dilated(inp, maxlen, filter_nums, embed_size):
    x = Reshape((maxlen, embed_size, 1))(inp)
    x = Conv2D(filter_nums[0], kernel_size=(2, embed_size), kernel_initializer='normal', padding="valid")(x)
    x = BatchNormalization()(x)
    x = Activation(activation="elu")(x)
    dilated_layers = [GlobalMaxPooling2D()(x)]
    x = Reshape((maxlen - 1, filter_nums[0]))(x)
    for num in filter_nums[1:]:
        x = Conv1D(num, kernel_size=2, kernel_initializer='normal', padding="causal")(x)
        x = BatchNormalization()(x)
        x = Activation(activation="elu")(x)
        dilated_layers.append(GlobalMaxPooling1D()(x))
    x = Concatenate(axis=-1)(dilated_layers)
    # x = Dropout(0.1)(x)
    # x = Dense(sum(filter_nums)//2)(x)
    # x = BatchNormalization()(x)
    # x = Activation(activation="elu")(x)
    return x



class MultiInpKerasClassifier(KerasClassifier):


    def fit(self, x, y, sample_weight=None, **kwargs):
        """Constructs a new model with `build_fn` & fit the model to `(x, y)`.

        # Arguments
            x : array-like, shape `(n_samples, n_features)`
                Training samples where `n_samples` is the number of samples
                and `n_features` is the number of features.
            y : array-like, shape `(n_samples,)` or `(n_samples, n_outputs)`
                True labels for `x`.
            **kwargs: dictionary arguments
                Legal arguments are the arguments of `Sequential.fit`

        # Returns
            history : object
                details about the training history at each epoch.

        # Raises
            ValueError: In case of invalid shape for `y` argument.
        """
        y = np.array(y)
        if len(y.shape) == 2 and y.shape[1] > 1:
            self.classes_ = np.arange(y.shape[1])
        elif (len(y.shape) == 2 and y.shape[1] == 1) or len(y.shape) == 1:
            self.classes_ = np.unique(y)
            y = np.searchsorted(self.classes_, y)
        else:
            raise ValueError('Invalid shape for y: ' + str(y.shape))
        self.n_classes_ = len(self.classes_)
        if sample_weight is not None:
            kwargs['sample_weight'] = sample_weight

        return super(KerasClassifier, self).fit(split_inp(parag_lens, x), y, **kwargs)

    def predict_proba(self, x, **kwargs):
        """Returns class probability estimates for the given test data.

        # Arguments
            x: array-like, shape `(n_samples, n_features)`
                Test samples where `n_samples` is the number of samples
                and `n_features` is the number of features.
            **kwargs: dictionary arguments
                Legal arguments are the arguments
                of `Sequential.predict_classes`.

        # Returns
            proba: array-like, shape `(n_samples, n_outputs)`
                Class probability estimates.
                In the case of binary classification,
                to match the scikit-learn API,
                will return an array of shape `(n_samples, 2)`
                (instead of `(n_sample, 1)` as in Keras).
        """
        kwargs = self.filter_sk_params(Sequential.predict_proba, kwargs)
        probs = self.model.predict(split_inp(parag_lens, x), **kwargs)

        # check if binary classification
        if probs.shape[1] == 1:
            # first column is probability of class 0 and second is of class 1
            probs = np.hstack([1 - probs, probs])
        return probs



In [7]:


def run(subfile='submission_char_word_cnn_adv', test_size=0.2, n_splits=4):
    """
    [CV]  batch_size=256, epochs=10, filter_sizes=[1, 2, 3, 4, 5], maxlen=200, nb_words=100000, num_filters=32, score=0.9774102756913097, total= 3.8min
    """
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    with tf.Session(config=config) as sess:
        #sess.run(tf.global_variables_initializer())
        cv_sets = ShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=0)
        parameters = {
            "dp": [0.2],
            "idp_size": [128, 64],
            "filter_sizes": [[1, 2, 3, 4, 5]],
            "filter_nums": [[30, 50, 80, 100]],
            "batch_size": [128, 64],
            "epochs": [3]
        }
        # define params used in fit() method here
        model = MultiInpKerasClassifier(build_fn=create_cnn, verbose=10, epochs=10, batch_size=256)
        grid = GridSearchCV(model, param_grid=parameters, scoring='roc_auc', cv=cv_sets, verbose=10, n_jobs=1)

        grid = grid.fit(X_train, y_train)
        best_clf = grid.best_estimator_
        print('CV score is {}'.format(grid.best_score_))
        print("best params are:")
        print(grid.best_params_)
        print(best_clf)

        y_pred = best_clf.predict_proba(X_test, batch_size=1024)
        print(y_pred.shape)
        submission[class_names] = y_pred
        submission.to_csv("subm/" + subfile + ".csv", index=False)






In [8]:
run()

Fitting 4 folds for each of 4 candidates, totalling 16 fits
[CV] batch_size=128, dp=0.2, epochs=3, filter_nums=[30, 50, 80, 100], filter_sizes=[1, 2, 3, 4, 5], idp_size=128 
(?, ?)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 250)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 400)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 250, 200)     4000200     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)    

Epoch 1/3
Epoch 2/3
Epoch 3/3
[CV]  batch_size=128, dp=0.2, epochs=3, filter_nums=[30, 50, 80, 100], filter_sizes=[1, 2, 3, 4, 5], idp_size=128, score=0.9842384160587452, total= 4.3min
[CV] batch_size=128, dp=0.2, epochs=3, filter_nums=[30, 50, 80, 100], filter_sizes=[1, 2, 3, 4, 5], idp_size=128 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.9min remaining:    0.0s


(?, ?)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 250)          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 400)          0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 250, 200)     4000200     input_3[0][0]                    
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 400, 100)     2000100     input_4[0][0]                    
__________________________________________________________________________________________________
res

Epoch 1/3
Epoch 2/3
Epoch 3/3
[CV]  batch_size=128, dp=0.2, epochs=3, filter_nums=[30, 50, 80, 100], filter_sizes=[1, 2, 3, 4, 5], idp_size=128, score=0.9860368093615172, total= 4.2min
[CV] batch_size=128, dp=0.2, epochs=3, filter_nums=[30, 50, 80, 100], filter_sizes=[1, 2, 3, 4, 5], idp_size=128 


...

Epoch 1/3
Epoch 2/3
Epoch 3/3
[CV]  batch_size=64, dp=0.2, epochs=3, filter_nums=[30, 50, 80, 100], filter_sizes=[1, 2, 3, 4, 5], idp_size=64, score=0.9830916186588055, total= 6.1min
[CV] batch_size=64, dp=0.2, epochs=3, filter_nums=[30, 50, 80, 100], filter_sizes=[1, 2, 3, 4, 5], idp_size=64 
(?, ?)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_31 (InputLayer)           (None, 250)          0                                            
__________________________________________________________________________________________________
input_32 (InputLayer)           (None, 400)          0                                            
__________________________________________________________________________________________________
embedding_31 (Embedding)        (None, 250, 200)     4000200     input_31[0][0]                   
_____

Epoch 1/3
Epoch 2/3
Epoch 3/3
[CV]  batch_size=64, dp=0.2, epochs=3, filter_nums=[30, 50, 80, 100], filter_sizes=[1, 2, 3, 4, 5], idp_size=64, score=0.9861032460120566, total= 6.2min


[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed: 95.4min finished


(?, ?)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_33 (InputLayer)           (None, 250)          0                                            
__________________________________________________________________________________________________
input_34 (InputLayer)           (None, 400)          0                                            
__________________________________________________________________________________________________
embedding_33 (Embedding)        (None, 250, 200)     4000200     input_33[0][0]                   
__________________________________________________________________________________________________
embedding_34 (Embedding)        (None, 400, 100)     2000100     input_34[0][0]                   
__________________________________________________________________________________________________
res

Epoch 1/3
Epoch 2/3
Epoch 3/3
CV score is 0.9857813379404369
best params are:
{'batch_size': 64, 'dp': 0.2, 'epochs': 3, 'filter_nums': [30, 50, 80, 100], 'filter_sizes': [1, 2, 3, 4, 5], 'idp_size': 128}
(153164, 6)
